# Data Collection

### Import required libraries

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
import os
import pandas as pd
from uuid import uuid1
from datetime import datetime
from datetime import date
import hashlib
from jetbot import Robot


### Display live camera feed

In [ ]:
image_resolution = 224

camera = Camera.instance(width=image_resolution, height=image_resolution, fps = 2)

image = widgets.Image(format='jpeg', width=image_resolution, height=image_resolution)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)


### Create directories + dataframe to store the data

In [ ]:
# Create an empty dataframe to store the data related to the images
image_param_column_list = ['UUID','DATE','TIME','HOLDING_POINT', 'EDGE ROTATION', 'EDGE DIST', 'EDGE ANGLE','SIDE',  'MARKING', ' LIGHT', 'LONG DIST', 'LAT DIST', 'ANGLE','SHA256']
image_param_df = pd.DataFrame(columns=image_param_column_list)

# Create the empty directories to store the data
holdingpoint_dir = 'dataset/holdingpoint'
noholdingpoint_dir = 'dataset/noholdingpoint'
hash_block_size = 65536

try:
    os.makedirs(noholdingpoint_dir)
    os.makedirs(holdingpoint_dir)
except FileExistsError:
    print('Directories not created because they already exist')

### Initialise variables

In [ ]:
longitudinal_distance_range = 60
lateral_distance_range = 60
axis_angle_range = 90
lateral_distance_step = 15
longitudinal_distance_step = 15
axis_angle_step = 22.5
longitudinal_distance = 0
lateral_distance = 0
axis_angle = 0

button_layout = widgets.Layout(width='128px', height='64px')
text_layout = widgets.Layout(width='200px', height='64px')

### Prepare Interface

In [ ]:
# General parameters setting interface

# Dropdown to select the marking colour
marking_selection = widgets.Dropdown(
    options=[('TRG1',0), ('TRG2',1),('TRG3',2), 
            ('TRG4',3), ('VAL1',4),('VAL2',5),
            ('TEST1',6), ('TEST2',7)],
    value=0,
    description='Marking:',
    disabled=False,
)

# Radiobutton to select the lighting conditions
lighting_selection = widgets.RadioButtons(
    options=[('Light',0), ('Dark',1)],
    description='Lighting:',
    disabled=False
)

# Radiobutton to select on which side of the robot the marking is located
side_selection = widgets.RadioButtons(
    options=[('LHS',0), ('RHS',1)],
    description='Holding point side:',
    disabled=False
)

# Radiobutton to indicate the rotation of the enclosure
edge_rotation_selection = widgets.RadioButtons(
    options=[('Initial',0), ('1/3 turn',1), ('2/3 turn',2)],
    description='Edge rotation:',
    disabled=False
)

# Radiobutton to indicate the distance between the marking en the enclosure's edge
edge_distance_selection = widgets.RadioButtons(
    options=[('Less than 25cm',0), ('More than 25cm',1)],
    description='Holding point distance from edge:',
    disabled=False
)

# Radiobutton to indicate the angle between the marking en the enclosure's edge
edge_angle_selection = widgets.RadioButtons(
    options=[('Parallel',0), ('Not parallel',1)],
    description='Holding point angle with edge:',
    disabled=False
)

# Grid parameters setting interface
longitudinal_distance_display = widgets.IntText(description='Longitudinal distance', layout=text_layout, value=longitudinal_distance)
lateral_distance_display = widgets.IntText(description='Lateral distance', layout=text_layout, value=lateral_distance)
axis_angle_display = widgets.IntText(description='Axis angle', layout=text_layout, value=axis_angle)

# Data collection buttons interface
noholdingpoint_button = widgets.Button(description='No holding point', button_style='success', layout=button_layout)
holdingpoint_button = widgets.Button(description='Holding point', button_style='danger', layout=button_layout)
pass_button = widgets.Button(description='Pass',button_style='info', layout=button_layout)

noholdingpoint_count = widgets.IntText(layout=button_layout, value=len(os.listdir(noholdingpoint_dir)))
holdingpoint_count = widgets.IntText(layout=button_layout, value=len(os.listdir(holdingpoint_dir)))


### Define functions

In [ ]:
# Define function to save the image to a directory
def save_snapshot(directory, fileid):
    image_path = os.path.join(directory, str(fileid) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

# Define function to compute the hash of a file
def generate_hash(directory, fileid):
    image_path = os.path.join(directory, str(fileid) + '.jpg')
    file_hash = hashlib.sha256()
    with open(image_path, 'rb') as f:
        fb = f.read(hash_block_size) 
        while len(fb) > 0: 
            file_hash.update(fb)
            fb = f.read(hash_block_size)
    return file_hash.hexdigest()  

# Define function to save the "operating parameters" to a dataframe
def save_parameters(fid, date, time, holdingpoint, edge_rotation, edge_dist, edge_angle, side, marking, lighting, long_dist, lat_dist, angle,fhash):
    global image_param_column_list, image_param_df
    temp_df = pd.DataFrame([[fid, date, time, holdingpoint, edge_rotation, edge_dist, edge_angle, side, marking, lighting, long_dist, lat_dist, angle,fhash]], columns=image_param_column_list)
    image_param_df = image_param_df.append(temp_df)
    pass

# Define a function to move on the grid pattern 
# (5 angles at each position, from 0 to 600mm every 150mm laterally and longitudinally)
def grid_pattern():
    global longitudinal_distance, lateral_distance, axis_angle
    cycle_completed = 0
    if axis_angle == axis_angle_range:
        axis_angle = 0
        if longitudinal_distance == longitudinal_distance_range:
            longitudinal_distance = 0
            if lateral_distance == lateral_distance_range:
                lateral_distance = 0
            else:
                lateral_distance = lateral_distance + lateral_distance_step
        else:
            longitudinal_distance = longitudinal_distance + longitudinal_distance_step
    else:
        axis_angle = axis_angle + axis_angle_step
    longitudinal_distance_display.value = longitudinal_distance
    lateral_distance_display.value = lateral_distance
    axis_angle_display.value = axis_angle

# Define function to save an image to the noholdingpoint directory
# and store the corresponding parameters in a dataframe
def save_noholdingpoint():
    global noholdingpoint_dir, noholdingpoint_count
    global edge_rotation_selection, edge_distance_selection, edge_angle_selection
    global side_selection, marking_selection, lighting_selection
    global longitudinal_distance, lateral_distance, axis_angle
    fileid = uuid1()
    date_now = str(date.today())
    time_now = datetime.now().strftime("%H:%M:%S")
    save_snapshot(noholdingpoint_dir, fileid)
    file_hash =  generate_hash(noholdingpoint_dir, fileid)
    save_parameters(fileid, date_now, time_now, 0, edge_rotation_selection.value, edge_distance_selection.value, edge_angle_selection.value, side_selection.value, marking_selection.value, lighting_selection.value, longitudinal_distance, lateral_distance, axis_angle, file_hash)
    noholdingpoint_count.value = len(os.listdir(noholdingpoint_dir))
    grid_pattern()

# Define function to save an image to the holdingpoint directory
# and store the corresponding parameters in a dataframe
def save_holdingpoint():
    global holdingpoint_dir, holdingpoint_count
    global edge_rotation_selection, edge_distance_selection, edge_angle_selection
    global side_selection, marking_selection, lighting_selection
    global longitudinal_distance, lateral_distance, axis_angle
    fileid = uuid1()
    date_now = str(date.today())
    time_now = datetime.now().strftime("%H:%M:%S")
    save_snapshot(holdingpoint_dir, fileid)
    file_hash =  generate_hash(holdingpoint_dir, fileid)
    save_parameters(fileid, date_now, time_now, 1, edge_rotation_selection.value, edge_distance_selection.value, edge_angle_selection.value, side_selection.value, marking_selection.value, lighting_selection.value, longitudinal_distance, lateral_distance, axis_angle, file_hash)
    holdingpoint_count.value = len(os.listdir(holdingpoint_dir))
    grid_pattern()
    
# Define function to "pass" (i.e. the picture cannot be taken for example if the grid pattern goes 
# outside of the enclosure...) and store the corresponding parameters in a dataframe
def save_pass():
    global marking_selection, lighting_selection
    global longitudinal_distance, lateral_distance, axis_angle
    date_now = str(date.today())
    time_now = datetime.now().strftime("%H:%M:%S")   
    save_parameters("NA", date_now, time_now, 0, edge_rotation_selection.value, edge_distance_selection.value, edge_angle_selection.value, side_selection.value, marking_selection.value, lighting_selection.value, longitudinal_distance, lateral_distance, axis_angle, "NA")
    grid_pattern()

# Define function to save an image to the noholdingpoint directory when using the remote control
# instead of the Jupyter Notebook interface
def save_noholding_point_ctrl(change):
    if change['new']:
        save_noholdingpoint()
    

### Set-up Controller

In [ ]:
# Set up remote control (gamepad)
controller = widgets.Controller(index=0)  
display(controller)

In [ ]:
# Create robot instance
robot = Robot()

# Define controller functions to move the robot
fwd_left_link = traitlets.dlink((controller.buttons[12], 'value'), (robot.left_motor, 'value'), transform=lambda x: x/8)
fwd_right_link = traitlets.dlink((controller.buttons[12], 'value'), (robot.right_motor, 'value'), transform=lambda x: x/8)

bck_left_link = traitlets.dlink((controller.buttons[13], 'value'), (robot.left_motor, 'value'), transform=lambda x: -x/8)
bck_right_link = traitlets.dlink((controller.buttons[13], 'value'), (robot.right_motor, 'value'), transform=lambda x: -x/8)

turn_left_left_link = traitlets.dlink((controller.buttons[14], 'value'), (robot.left_motor, 'value'), transform=lambda x: -x/8)
turn_left_right_link = traitlets.dlink((controller.buttons[14], 'value'), (robot.right_motor, 'value'), transform=lambda x: x/8)

turn_right_left_link = traitlets.dlink((controller.buttons[15], 'value'), (robot.left_motor, 'value'), transform=lambda x: x/8)
turn_right_right_link = traitlets.dlink((controller.buttons[15], 'value'), (robot.right_motor, 'value'), transform=lambda x: -x/8)

In [ ]:
# Define button to save image in noholdingpont folder
controller.buttons[7].observe(save_noholding_point_ctrl, names='value')

### Collect data

In [ ]:
# Display all controls
display(widgets.HBox([marking_selection, edge_rotation_selection]))
display(widgets.HBox([lighting_selection, edge_distance_selection]))
display(widgets.HBox([side_selection,edge_angle_selection]))

display(widgets.HBox([longitudinal_distance_display, lateral_distance_display]))
display(widgets.HBox([axis_angle_display]))

display(widgets.HBox([holdingpoint_button, holdingpoint_count, pass_button]))
display(widgets.HBox([noholdingpoint_button,noholdingpoint_count]))

display(image, width = image_resolution, height = image_resolution)

# Save images and information in .csv file depending on which button is pressed by operator 
noholdingpoint_button.on_click(lambda x: save_noholdingpoint())
holdingpoint_button.on_click(lambda x: save_holdingpoint())
pass_button.on_click(lambda x: save_pass())




### Save Image parameters to dedicated .csv file

In [ ]:
# Save content of dataframe (i.e. all operating parameters for all pictures) to a .csv file
now = datetime.now().strftime("%d%m%Y_%H%M%S")
image_param_df.to_csv('dataset/Image_Parameters_' + now + '.csv')

image_param_file_hash = hashlib.sha512()
with open('dataset/Image_Parameters_' + now + '.csv', 'rb') as f:
    fb = f.read(hash_block_size) 
    while len(fb) > 0: 
        image_param_file_hash.update(fb)
        fb = f.read(hash_block_size)
image_param_file_hash_file = open('dataset/Image_Parameters_' + now + '_hash.txt', "w")
image_param_file_hash_file.write(image_param_file_hash.hexdigest())
image_param_file_hash_file.close()
        

### Zip dataset folder for export

In [ ]:
# Generate zip file of images and .csv file for export
!zip -r -q dataset.zip dataset


In [ ]:
# Compute hash for overall .csv file
dataset_hash = hashlib.sha512()
with open("dataset.zip", 'rb') as f:
    fb = f.read(hash_block_size) 
    while len(fb) > 0: 
        dataset_hash.update(fb)
        fb = f.read(hash_block_size)
dataset_hash.hexdigest()

dataset_hash_file = open("dataset_hash.txt", "w")
dataset_hash_file.write(dataset_hash.hexdigest())
dataset_hash_file.close()

In [ ]:
# Colse al conections with robot
camera_link.unlink()
fwd_left_link.unlink()
fwd_right_link.unlink()
bck_left_link.unlink()
bck_right_link.unlink()
turn_left_left_link.unlink()
turn_left_right_link.unlink()
turn_right_left_link.unlink()
turn_right_right_link.unlink()
